In [1]:
from utils import *
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

import tensorflow.keras as keras
import tensorflow.keras.backend as K

import os
import os.path as osp
import sys
import json

import numpy as np
#from scipy import linalg as LA

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
# from colorspacious import cspace_converter
from collections import OrderedDict


import tensorflow.keras as keras
import tensorflow.keras.backend as K

import numpy as np
import pandas

2022-08-02 15:35:24.442888: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


1 Physical GPUs, 1 Logical GPUs


2022-08-02 15:35:25.613479: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-02 15:35:25.614264: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-02 15:35:25.650007: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:41:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-08-02 15:35:25.650045: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-02 15:35:25.651786: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-08-02 15:35:25.651816: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-0

In [7]:
%cd '/global/home/users/yifengh3/VAE/EMD_VAE'

/global/home/users/yifengh3/VAE/EMD_VAE


In [9]:
import utils.VAE_model_tools_param
# from utils.VAE_model_tools_param import build_and_compile_annealing_vae, betaVAEModel, reset_metrics, loss_tracker, myTerminateOnNaN

ModuleNotFoundError: No module named 'utils.VAE_model_tools_param'; 'utils' is not a package

In [ ]:
fn =  '/global/home/users/yifengh3/VAE/vec_data/B_background.h5'
vae_args_file = "/global/home/users/yifengh3/VAE/vec_data/vec_model_1/vae_args.dat"

df = pandas.read_hdf(fn,stop=1000000)
print(df.shape)
print("Memory in GB:",sum(df.memory_usage(deep=True)) / (1024**3)+sum(df.memory_usage(deep=True)) / (1024**3))

# Data file contains, for each event, 50 particles (with zero padding), each particle with pT, eta, phi, E.
# we only need first 200th entries, weight is not required here
numparts=50
data = df.values[:,:200]
print(data.shape)
data = data.reshape((-1,numparts,4))
data = data.astype(float)

# Normalize pTs so that HT = 1
HT = np.sum(data[:,:,0],axis=-1)
data[:,:,0] = data[:,:,0]/HT[:,None]
data[:,:,-1] = data[:,:,-1]/HT[:,None]

# Inputs x to NN will be: pT, eta, cos(phi), sin(phi), log E
# Separated phi into cos and sin for continuity around full detector, so make things easier for NN.
# Also adding the log E is mainly because it seems like it should make things easier for NN, since there is an exponential spread in particle energies.
# Feel free to change these choices as desired. E.g. px, py might be equally as good as pt, sin, cos.
sig_input = np.zeros((len(data),numparts,4))
sig_input[:,:,:2] = data[:,:,:2]
sig_input[:,:,2] = np.cos(data[:,:,2])
sig_input[:,:,3] = np.sin(data[:,:,2])
#sig_input[:,:,4] = np.log(data[:,:,3]+1e-8)


data_x = sig_input
# Event 'labels' y are [pT, eta, phi], which is used to calculate EMD to output which is also pT, eta, phi.
data_y = data[:,:,:3]

numtrain = 500000
train_x = data_x[:numtrain]
train_y = data_y[:numtrain]
valid_x = data_x[numtrain:numtrain+100000]
valid_y = data_y[numtrain:numtrain+100000]

# train_output_dir = osp.join(model_dir,"end_beta_checkpoint") #create_dir(osp.join(output_dir, experiment_name))
train_output_dir = model_dir


with open(vae_args_file,'r') as f:
     vae_arg_dict = json.loads(f.read())

print("\n\n vae_arg_dict:", vae_arg_dict)

vae, encoder, decoder = build_and_compile_annealing_vae(**vae_arg_dict)

In [ ]:
logbetas = np.concatenate((np.logspace(-5.,1.,13),
                          np.logspace(1.,-1.,5)[1:],
                          np.logspace(-1.,1.,5)[1:],
                          np.logspace(1.,-1.,5)[1:],
                          np.logspace(-1.,0.,3)[1:]))

In [ ]:
model_dir = "/global/home/users/yifengh3/VAE/vec_data/vec_model_1"
model_name = "model_weights_end_1491_betar_1.0e+00.hdf5"

In [ ]:
vae.load_weights(os.path.join(model_dir,model_name))

In [ ]:
result_D = np.zeros(len(logbetas))
batchsize = 1000
nsamples = 1
numbatches = 5
for i in tqdm(range(numbatches)):
    if i%10 == 0:
        print("Processing batch", i, "of", numbatches)
    x = valid_x[i*batchsize:(i+1)*batchsize]
    y = valid_y[i*batchsize:(i+1)*batchsize]
    result_D += [np.mean(vae.heat_capacity_D([[x,np.ones(len(x))*logbeta],y],nsamples=nsamples)) for logbeta in logbetas]
result_D = result_D / numbatches
plt.plot(np.power(10,logbetas),result_D)
plt.semilogx()
plt.ylim([0,10])
plt.show()